# Entrenamiento LoRA en Google Colab (Qwen 3 4B)

Este notebook prepara datos desde Hugging Face y entrena un adapter LoRA para generar ejercicios de Jupyter.


## 1) Clonar repo e instalar dependencias


In [ ]:
!git clone https://github.com/TU_USUARIO/TU_REPO.git
%cd TU_REPO/jupyter-notebook-backend/python/ia-jupyterNotebook
!python -m pip install --upgrade pip
!pip install -r requirements-finetune.txt


## 2) Preparar dataset (streaming, rapido)

Tip: deja `--include-packages ""` para evitar que el archivo salga vacio.


In [ ]:
!python prepare_hf_dataset.py \
  --dataset jupyter-agent/jupyter-agent-dataset \
  --split non_thinking \
  --streaming \
  --output hf_jupyter_messages.jsonl \
  --max-samples 4000 \
  --include-packages ""

!wc -l hf_jupyter_messages.jsonl
!head -n 2 hf_jupyter_messages.jsonl


## 3) Entrenar LoRA (perfil estable)

Si tienes GPU grande (A100/L4), puedes subir `--max-seq-length` y `--max-steps`.


In [ ]:
!PYTORCH_ALLOC_CONF=expandable_segments:True python train_lora_qwen.py \
  --model-name Qwen/Qwen3-4B-Instruct-2507 \
  --train-file hf_jupyter_messages.jsonl \
  --output-dir qwen3-jupyter-lora \
  --max-steps 300 \
  --max-seq-length 256 \
  --per-device-train-batch-size 1 \
  --gradient-accumulation-steps 16 \
  --lora-r 4 \
  --lora-alpha 8


## 4) Empaquetar y descargar adapter

Esto genera un zip con el modelo LoRA para llevarlo a tu proyecto local.


In [ ]:
!zip -r qwen3-jupyter-lora.zip qwen3-jupyter-lora


In [ ]:
from google.colab import files
files.download('qwen3-jupyter-lora.zip')


## 5) Uso local

1. Copia la carpeta `qwen3-jupyter-lora` a `jupyter-notebook-backend/python/ia-jupyterNotebook/`.
2. Levanta tu API Python:

```bash
cd jupyter-notebook-backend/python/ia-jupyterNotebook
source .venv/bin/activate
PYTORCH_ALLOC_CONF=expandable_segments:True MAX_NEW_TOKENS=96 uvicorn main:app --host 0.0.0.0 --port 8001
```
